In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

# Load the dataset
df = pd.read_csv('1654342Pet shop.csv')
df.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size
0,1,82640,toys,off_brand,3.6,11,30.0,76.4,868
1,2,194000,toys,off_brand,4.0,11,15.0,57.1,2625
2,3,148423,food,off_brand,4.5,5,28.0,59.8,974
3,4,201070,toys,off_brand,4.7,2,15.0,33.6,3107
4,5,170240,food,off_brand,4.6,1,NaN,40.7,2384


In [33]:
product_category_dummies = pd.get_dummies(df['product_category'])
df_product_category_dummies = pd.concat([df,product_category_dummies],axis=1)
df_product_category_dummies.isnull().sum()




Unnamed: 0            0
products_sold         0
product_category      0
quality             111
satisfaction          0
discount              0
retail_price        154
perc_physical        69
market_size           0
food                  0
health                0
other                 0
toys                  0
dtype: int64

In [34]:
filtered_df = df_product_category_dummies.dropna()
filtered_df.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys
0,1,82640,toys,off_brand,3.6,11,30.0,76.4,868,False,False,False,True
1,2,194000,toys,off_brand,4.0,11,15.0,57.1,2625,False,False,False,True
2,3,148423,food,off_brand,4.5,5,28.0,59.8,974,True,False,False,False
3,4,201070,toys,off_brand,4.7,2,15.0,33.6,3107,False,False,False,True
5,6,219676,other,premium,4.6,7,25.0,44.7,3735,False,False,True,False


In [30]:
model_no_changes = smf.ols('products_sold ~ food + health + other + toys + quality + satisfaction + discount + retail_price + perc_physical + market_size',data = df_product_category_dummies).fit()
print(model_no_changes.summary())
model_no_missing_values = smf.ols('products_sold ~ food + health + other + toys + quality + satisfaction + discount + retail_price + perc_physical + market_size',data = df_product_category_dummies).fit()
print(model_no_changes.summary())

                            OLS Regression Results                            
Dep. Variable:          products_sold   R-squared:                       0.782
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     767.4
Date:                Wed, 27 Mar 2024   Prob (F-statistic):               0.00
Time:                        15:29:52   Log-Likelihood:                -22268.
No. Observations:                1937   AIC:                         4.456e+04
Df Residuals:                    1927   BIC:                         4.461e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -8.255e+04   4034

In [32]:
CooksD = model_no_changes.get_influence().cooks_distance
n = len(filtered_df)
filtered_df['outlier'] = CooksD[0] > 4/n
filtered_df.head()

/tmp/ipykernel_8373/1243585742.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['outlier'] = CooksD[0] > 4/n


,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,outlier
0,1,82640,toys,off_brand,3.6,11,30.0,76.4,868,False,False,False,True,False
1,2,194000,toys,off_brand,4.0,11,15.0,57.1,2625,False,False,False,True,False
2,3,148423,food,off_brand,4.5,5,28.0,59.8,974,True,False,False,False,False
3,4,201070,toys,off_brand,4.7,2,15.0,33.6,3107,False,False,False,True,False
5,6,219676,other,premium,4.6,7,25.0,44.7,3735,False,False,True,False,False


,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,outlier
0,1,82640,toys,off_brand,3.6,11,30.0,76.4,868,False,False,False,True,False
1,2,194000,toys,off_brand,4.0,11,15.0,57.1,2625,False,False,False,True,False
2,3,148423,food,off_brand,4.5,5,28.0,59.8,974,True,False,False,False,False
3,4,201070,toys,off_brand,4.7,2,15.0,33.6,3107,False,False,False,True,False
5,6,219676,other,premium,4.6,7,25.0,44.7,3735,False,False,True,False,False
